In [1]:
import pandas as pd

filmes = pd.read_csv("movies.csv")
filmes.columns = ["filmeId", "titulo", "generos"]
filmes = filmes.set_index("filmeId")
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [2]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# Primeira tentativa de recomendação: heurística de total de votos

In [4]:
total_de_votos = notas["filmeId"].value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [5]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,titulo,generos,total_de_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [6]:
filmes.sort_values("total_de_votos", ascending = False).head(10)

,titulo,generos,total_de_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


In [7]:
notas_medias = notas.groupby("filmeId").mean()["nota"]
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

In [8]:
filmes["nota_media"] = notas_medias
filmes.sort_values("total_de_votos", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


# Uma segunda heurística: nota média e filtrando votos

In [9]:
filmes.sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [10]:
filmes.query("total_de_votos >= 10").sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
1041,Secrets & Lies (1996),Drama,11.0,4.590909
3451,Guess Who's Coming to Dinner (1967),Drama,11.0,4.545455
1178,Paths of Glory (1957),Drama|War,12.0,4.541667
1104,"Streetcar Named Desire, A (1951)",Drama,20.0,4.475000
2360,"Celebration, The (Festen) (1998)",Drama,12.0,4.458333
1217,Ran (1985),Drama|War,15.0,4.433333
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
951,His Girl Friday (1940),Comedy|Romance,14.0,4.392857
1927,All Quiet on the Western Front (1930),Action|Drama|War,10.0,4.350000


In [11]:
filmes_com_mais_de_50_votos = filmes.query("total_de_votos >= 50")
filmes_com_mais_de_50_votos.sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [12]:
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [13]:
aventura_infantil_e_fantasia = filmes_com_mais_de_50_votos.query("generos=='Adventure|Children|Fantasy'")
aventura_infantil_e_fantasia.drop(eu_assisti, errors='ignore').sort_values("nota_media", ascending = False)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548


# Procura usuários Similares

In [14]:
joao = [4, 4.5]
maria = [5, 5]

import matplotlib.pyplot as plt

plt.plot(4, 4.5, "go")
plt.plot(5, 5, "yo")
plt.plot(3.5, 4.5, "yo")
plt.legend(["João", "Maria", "Joaquina"])
plt.title("Distância entre usuários")
plt.plot([4, 5], [4.5, 4.5], color="b", linestyle="-")
plt.plot([4, 5], [4.5, 5], color="b", linestyle="-")
plt.plot([5, 5], [4.5, 5], color="b", linestyle="-")
plt.show()

<Figure size 640x480 with 1 Axes>

In [15]:
import numpy as np
joao = np.array([4, 4.5])
maria = np.array([5, 5])
joaquina = np.array([3.5, 4.5])

joao - maria

array([-1. , -0.5])

In [16]:
from math import sqrt
def pitagoras(a, b):
    (delta_x, delta_y) = a - b
    return sqrt(delta_x * delta_x + delta_y * delta_y)

pitagoras(joao, maria)

1.118033988749895

In [17]:
np.linalg.norm(joao - maria)

1.118033988749895

In [18]:
def pitagoras(a, b):
    return np.linalg.norm(a - b)
pitagoras(joao, maria)

1.118033988749895

In [19]:
print(pitagoras(joao, maria))

1.118033988749895


In [20]:
print(pitagoras(joao, joaquina))

0.5


In [21]:
def distancia(a, b):
    return np.linalg.norm(a - b)

In [22]:
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [23]:
notas_usuario = notas.query("usuarioId == 1")
notas_usuario = notas_usuario[["filmeId", "nota"]].set_index('filmeId')
# notas_usuario.loc[]

In [24]:
def notas_do_usuario(usuario):
    notas_usuario = notas.query("usuarioId == %d" % usuario)
    notas_usuario = notas_usuario[["filmeId", "nota"]].set_index('filmeId')
    return notas_usuario

In [25]:
usuario1 = notas_do_usuario(41)
usuario2 = notas_do_usuario(32)

In [26]:
usuario1.head()

,nota
filmeId,
47,3.5
50,3.0
111,2.0
215,2.0
288,3.0


In [27]:
usuario2.head()

,nota
filmeId,
1,3.0
3,3.0
6,3.0
7,4.0
21,4.0


In [28]:
diferencas = usuario1.join(usuario2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()

In [29]:
diferencas

,nota_esquerda,nota_direita
filmeId,,
50,3.0,5.0
296,5.0,5.0
357,4.0,3.0
593,4.0,5.0
608,4.0,5.0


# Distância entre usuários do Dataset

In [35]:
def distancia_de_vetores(a, b):
    return np.linalg.norm(a - b)

In [36]:
diferencas = usuario1.join(usuario2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
distancia_de_vetores(diferencas["nota_esquerda"], diferencas["nota_direita"])

2.6457513110645907

In [43]:
def distacia_de_usuarios(u1, u2):
    notas_1 = notas_do_usuario(u1)
    notas_2 = notas_do_usuario(u2)
    diferencas = notas_1.join(notas_2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    diferencas = distancia_de_vetores(diferencas["nota_esquerda"], diferencas["nota_direita"])
    return [u1, u2, diferencas]

In [44]:
distacia_de_usuarios(41, 32)

[41, 32, 2.6457513110645907]

In [45]:
qtd_de_usuarios = len(notas['usuarioId'].unique())

In [46]:
notas['usuarioId'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [48]:
qtd_de_usuarios
print('Temos {0} usuários'.format(qtd_de_usuarios))

Temos 610 usuários


In [49]:
for usuario_id in notas['usuarioId'].unique():
    print(usuario_id)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [52]:
voce_id = 1
distancias = []
for usuario_id in notas['usuarioId'].unique():
    if(voce_id == usuario_id): continue
    informacoes = distacia_de_usuarios(voce_id, usuario_id)
    distancias.append(informacoes)

In [53]:
distancias[:5]

[[1, 2, 1.4142135623730951],
 [1, 3, 8.200609733428363],
 [1, 4, 11.135528725660043],
 [1, 5, 3.7416573867739413],
 [1, 6, 8.602325267042627]]

In [55]:
def distancia_de_todos(user):
    distancias = []
    for usuario_id in notas['usuarioId'].unique():
        if(user == usuario_id): continue
        informacoes = distacia_de_usuarios(user, usuario_id)
        distancias.append(informacoes)
    return distancias

In [56]:
distancia_de_todos(2)[:5]

[[2, 1, 1.4142135623730951],
 [2, 3, 0.0],
 [2, 4, 3.5],
 [2, 5, 0.0],
 [2, 6, 2.23606797749979]]

In [61]:
def distancia_de_todos(user):
    todos_usuarios = notas['usuarioId'].unique()
    distancias = [distacia_de_usuarios(user, usuario_id) for usuario_id in todos_usuarios if(user != usuario_id)]
    distancias = pd.DataFrame(distancias, columns = ["Você", "Outra Pessoa", "Distância"])
    return distancias

In [63]:
distancia_de_todos(1)

,Você,Outra Pessoa,Distância
0,1,2,1.414214
1,1,3,8.200610
2,1,4,11.135529
3,1,5,3.741657
4,1,6,8.602325
5,1,7,7.106335
6,1,8,3.741657
7,1,9,1.000000
8,1,10,7.858117
9,1,11,4.242641


In [67]:
notas_do_usuario(1).join(notas_do_usuario(2))

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
70,3.0
101,5.0
110,4.0
151,5.0


In [68]:
notas_do_usuario(589)

,nota
filmeId,
25,5.0
36,5.0
111,5.0
150,4.0
161,3.0
185,3.0
193,3.0
231,4.0
235,4.0


In [72]:
notas_do_usuario(1).join(notas_do_usuario(2), lsuffix="_1", rsuffix="_2").dropna()

,nota_1,nota_2
filmeId,,
333,5.0,4.0
3578,5.0,4.0


# Usuários sem nada em comum são colocados bem distantes um do outro

In [94]:
def distacia_de_usuarios(u1, u2, minimo = 5):
    notas_1 = notas_do_usuario(u1)
    notas_2 = notas_do_usuario(u2)
    diferencas = notas_1.join(notas_2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    if(len(diferencas) < minimo): return [u1, u2, 100000]
    diferencas = distancia_de_vetores(diferencas["nota_esquerda"], diferencas["nota_direita"])
    return [u1, u2, diferencas]

In [95]:
def distancia_de_todos(user):
    todos_usuarios = notas['usuarioId'].unique()
    distancias = [distacia_de_usuarios(user, usuario_id) for usuario_id in todos_usuarios if(user != usuario_id)]
    distancias = pd.DataFrame(distancias, columns = ["Você", "Outra Pessoa", "Distância"])
    return distancias

In [96]:
distancia_de_todos(1)

,Você,Outra Pessoa,Distância
0,1,2,100000.000000
1,1,3,8.200610
2,1,4,11.135529
3,1,5,3.741657
4,1,6,8.602325
5,1,7,7.106335
6,1,8,3.741657
7,1,9,1.000000
8,1,10,7.858117
9,1,11,4.242641


In [99]:
def mais_proximos(user):
    distancias = distancia_de_todos(user)
    distancias = distancias.sort_values("Distância")
    return distancias

In [100]:
mais_proximos(1).head(10)

,Você,Outra Pessoa,Distância
75,1,77,0.000000
509,1,511,0.500000
364,1,366,0.707107
256,1,258,1.000000
47,1,49,1.000000
521,1,523,1.000000
7,1,9,1.000000
317,1,319,1.118034
396,1,398,1.224745
63,1,65,1.322876


In [101]:
notas_do_usuario(1).join(notas_do_usuario(77), lsuffix="_1", rsuffix="_2")

,nota_1,nota_2
filmeId,,
1,4.0,NaN
3,4.0,NaN
6,4.0,NaN
47,5.0,NaN
50,5.0,NaN
70,3.0,NaN
101,5.0,NaN
110,4.0,NaN
151,5.0,NaN


# Parâmetros para teste

In [109]:
def distacia_de_usuarios(u1, u2, minimo = 5):
    notas_1 = notas_do_usuario(u1)
    notas_2 = notas_do_usuario(u2)
    diferencas = notas_1.join(notas_2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    if(len(diferencas) < minimo): return None
    diferenca = distancia_de_vetores(diferencas["nota_esquerda"], diferencas["nota_direita"])
    return [u1, u2, diferenca]

def distancia_de_todos(user, n = None):
    todos_usuarios = notas['usuarioId'].unique()
    if n:
        todos_usuarios = todos_usuarios[:n]
    distancias = [distacia_de_usuarios(user, usuario_id) for usuario_id in todos_usuarios if(user != usuario_id)]
    distancias = list(filter(None, distancias))
    distancias = pd.DataFrame(distancias, columns = ["voce", "outra_pessoa", "distancia"])
    return distancias

def mais_proximos(user, n = None):
    distancias = distancia_de_todos(user, n = n)
    distancias = distancias.sort_values("distancia")
    return distancias

In [111]:
mais_proximos(1, 50)

,voce,outra_pessoa,distancia
45,1,49,1.000000
6,1,9,1.000000
21,1,25,1.414214
9,1,13,1.414214
26,1,30,1.802776
22,1,26,2.236068
31,1,35,2.236068
42,1,46,3.316625
2,1,5,3.741657
5,1,8,3.741657
